The very first step is to download and install the FESAPI wheel : pip install fesapi

Once FESAPI wheel installed, you need to first import it in order to be able to call its API (https://www.f2i-consulting.com/fesapi/doxygen/)

In [1]:
import fesapi

Let's now create a DataObjectRepository in order to store Energistics dataobjects in-memory.

In [2]:
repo = fesapi.DataObjectRepository()

Energistics local CRS and Energistics HDF Proxy dataobjects are respectively required when we want to export CRS aware dataobjects and RESQML2 representations. Let's create them and set them as default ones for all other Energistics dataobject we will create later.

In [3]:
# Create an unknown local depth oriented 3D CRS in meters with no offset and no rotation
local_3d_crs = repo.createLocalDepth3dCrs("", "Default CRS",
                                          0.0, 0.0, 0.0, 0.0,
                                          fesapi.eml20__LengthUom_m, 5215,
                                          fesapi.eml20__LengthUom_m, "Unknown",
                                          False)
repo.setDefaultCrs(local_3d_crs)
# Create an HDF Proxy to store large numerical value arrays in an HDF5 file located in the current folder nad named TestingFesapiWithPython.h5
file_name_without_extension = "TestingFesapiWithPython" 
hdf_proxy = repo.createHdfProxy("", "Hdf Proxy",
                                ".", file_name_without_extension + ".h5",
                                fesapi.DataObjectRepository.openingMode_OVERWRITE)
repo.setDefaultHdfProxy(hdf_proxy)

We can now create plenty of Energistics dataobjects. Let's try a Horizon 2d grid.

In [4]:
horizon_feature = repo.createHorizon("", "My horizon feature")
horizon_interpretation = repo.createHorizonInterpretation(horizon_feature, "", "My horizon interpretation")
horizon_grid_2d_representation = repo.createGrid2dRepresentation(horizon_interpretation, "", "My horizon representation")

And let's set a 2x3 geometry on this 2d grid where the Z values equal hundred times the node index.

The first (aka I) axis of this 2d grid is aligned on X and the second (aka J) axis of this 2d grid is aligned on Y.

The spacing on the first axis between two node is 25m and the spacing on the second axis between two node is 50m

In [5]:
resqml_points = fesapi.DoubleArray(24)
for i in range(6):
    resqml_points.setitem(i, i*100)
horizon_grid_2d_representation.setGeometryAsArray2dOfExplicitZ(resqml_points, 2, 3, hdf_proxy,
                                                                0.0, 0.0, 0.0,
                                                               1.0, 0.0, 0.0, 25.0,
                                                               0.0, 1.0, 0.0, 50.0)

We can now serialize our DataObjectRepository into an EPC document in order to share it with other EPC aware application.

In [6]:
# Create an EPC Document in the current folder nad named TestingFesapiWithPython.epc
epc_file = fesapi.EpcDocument(file_name_without_extension + ".epc")
epc_file.serializeFrom(repo)
epc_file.close()